<a href="https://colab.research.google.com/github/manishmcsa/Assigment-6/blob/main/model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm

In [ ]:
def test(model, device, test_loader, test_losses, test_acc, misclassified_imgs,
         is_last_epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target,
                                    reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1,
                                 keepdim=True)  # get the index of the max
            # log-probability
            is_correct = pred.eq(target.view_as(pred))
            if is_last_epoch:
                misclassified_inds = (is_correct == 0).nonzero()[:, 0]
                for mis_ind in misclassified_inds:
                    if len(misclassified_imgs) == 25:
                        break
                    misclassified_imgs.append({
                        "target": target[mis_ind].cpu().numpy(),
                        "pred": pred[mis_ind][0].cpu().numpy(),
                        "img": data[mis_ind].cpu().numpy()[0]
                    })
            correct += is_correct.sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))
